In [156]:
import torch
import numpy as np
from sketched_optimizer import SketchedSGD, SketchedSum, SketchedModel

In [157]:
X = torch.randn(1000, 3)
w = torch.tensor([[.1],[.2],[.3]])

In [158]:
y = torch.matmul(X,w)

In [159]:
y.shape

torch.Size([1000, 1])

In [160]:
model = torch.nn.Sequential(torch.nn.Linear(3, 1))
model = SketchedModel(model)

In [162]:
opt = torch.optim.SGD(model.parameters(), lr=0.001)
opt = SketchedSGD(opt, k=0, accumulateError=True, p1=0, p2=4)

In [163]:
summer = SketchedSum(opt, c=20, r=5, numWorkers=4)

device cpu
D: 4
sketchMask.sum(): 3
making sketches


In [153]:
yPred = model(X)
loss = (yPred - y)**2
loss = summer(loss)
loss.item()

0.0

In [164]:
for i in range(20):
    opt.zero_grad()

    yPred = model(X)
    loss = (yPred - y)**2

    loss = summer(loss)

    print("[{}] Loss: {}".format(i, loss.item()))

    loss.backward()

    opt.step()

[0] Loss: 1.3317620754241943
[1] Loss: 1.3317620754241943
[2] Loss: 1.3317620754241943
[3] Loss: 1.3317620754241943
[4] Loss: 1.3317620754241943
[5] Loss: 1.3317620754241943
[6] Loss: 1.3317620754241943
[7] Loss: 1.3317620754241943
[8] Loss: 1.3317620754241943
[9] Loss: 1.3317620754241943
[10] Loss: 1.3317620754241943
[11] Loss: 1.3317620754241943
[12] Loss: 1.3317620754241943
[13] Loss: 1.3317620754241943
[14] Loss: 1.3317620754241943
[15] Loss: 1.3317620754241943
[16] Loss: 1.3317620754241943
[17] Loss: 1.3317620754241943
[18] Loss: 1.3317620754241943
[19] Loss: 1.3317620754241943


In [161]:
state_dict = model.state_dict()
state_dict['0.weight'] = torch.tensor([[0.11,0.22,0.33]])
state_dict['0.bias'] = torch.tensor([0.0])
model.load_state_dict(state_dict)

<All keys matched successfully>